In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
sales = pd.read_excel('Coffee_Shop_Sales.xlsx')
costs = pd.read_excel('Coffee_cost.xlsx', sheet_name='cost')

#Join table
sales_cost = pd.merge(sales, costs, left_on='product_detail', right_on='Item', how='inner')

In [3]:
print(sales_cost.dtypes)
print(sales_cost.isna().sum())

Unnamed: 0                   int64
transaction_id               int64
transaction_date    datetime64[ns]
transaction_time            object
transaction_qty              int64
store_id                     int64
store_location              object
product_id                   int64
unit_price                 float64
product_category            object
product_type                object
product_detail              object
Item                        object
Cost                       float64
dtype: object
Unnamed: 0          0
transaction_id      0
transaction_date    0
transaction_time    1
transaction_qty     0
store_id            0
store_location      3
product_id          0
unit_price          3
product_category    0
product_type        0
product_detail      0
Item                0
Cost                0
dtype: int64


In [4]:
# Missing values in store_location
# Extract unique pairs of store_id and store_location
unique_stores = sales_cost[['store_id', 'store_location']].drop_duplicates()
print(unique_stores)

       store_id   store_location
0             8   Hell's Kitchen
1             5  Lower Manhattan
4             3          Astoria
25798         3              NaN


In [5]:
# fill the Null values based on each store_id

# make a dict only containing correct store_id and location
store_map = sales_cost.dropna(subset=['store_location']).set_index('store_id')['store_location'].to_dict()

# Fill missing values based on the store_map
sales_cost['store_location'] = sales_cost['store_id'].map(store_map)

# check 
unique_stores = sales_cost[['store_id', 'store_location']].drop_duplicates()
print(unique_stores)

   store_id   store_location
0         8   Hell's Kitchen
1         5  Lower Manhattan
4         3          Astoria


In [6]:
# do the same thing to the unit_price and product_id
unit_map = sales_cost.dropna(subset='unit_price').set_index('product_id')['unit_price'].to_dict()
sales_cost['unit_price'] = sales_cost['product_id'].map(unit_map)

# transaction_time and qty are not possible to tell the exact value and there is only one missing value respectively. Therefore drop these missing values. 
sales_cost = sales_cost.dropna()

print(sales_cost.isna().sum())

Unnamed: 0          0
transaction_id      0
transaction_date    0
transaction_time    0
transaction_qty     0
store_id            0
store_location      0
product_id          0
unit_price          0
product_category    0
product_type        0
product_detail      0
Item                0
Cost                0
dtype: int64


In [7]:
# Add Sales and profits columns
sales_cost['sales'] = sales_cost['transaction_qty'] * sales_cost['unit_price']
sales_cost['merge'] = sales_cost['unit_price'] - sales_cost['Cost']
sales_cost['profit'] = (sales_cost['unit_price'] - sales_cost['Cost']) * sales_cost['transaction_qty'] 

In [8]:
# Total profits by each category
category_profits = sales_cost.groupby('product_category').agg({'profit': 'sum','transaction_id': 'count'}).reset_index()
print(category_profits)

      product_category    profit  transaction_id
0               Bakery  28469.65           12532
1              Branded   6132.00             452
2                Coffe      2.45               1
3               Coffee  77058.55           35052
4         Coffee beans  16125.35             914
5   Drinking Chocolate  20558.60            6846
6             Flavours   3759.00            4048
7            Loose Tea   5841.20             732
8   Packaged Chocolate   1788.05             293
9                  Tea  55342.85           27312
10                 Tee      2.20               1


In [9]:
# Replace typo to correct spell: Coffe and Tee
sales_cost['product_category'] = sales_cost['product_category'].replace({'Coffe':'Coffee', 'Tee': 'Tea'})

# check
category_profits = sales_cost.groupby('product_category').agg({'profit': 'sum','transaction_id': 'count'}).reset_index()
print(category_profits)

     product_category    profit  transaction_id
0              Bakery  28469.65           12532
1             Branded   6132.00             452
2              Coffee  77061.00           35053
3        Coffee beans  16125.35             914
4  Drinking Chocolate  20558.60            6846
5            Flavours   3759.00            4048
6           Loose Tea   5841.20             732
7  Packaged Chocolate   1788.05             293
8                 Tea  55345.05           27313


In [13]:
# Total profits by each store
store_profits = sales_cost.groupby('store_id').agg({'profit': 'sum','transaction_id': 'count'}).reset_index()
print(store_profits)

   store_id    profit  transaction_id
0         3  70085.99           29841
1         5  72413.79           28419
2         8  72580.12           29923


In [11]:
weather = pd.read_excel('Store_Weather_Data_2023.xlsx')
# check data type 
weather.dtypes
weather['transaction_date'].astype('datetime64[ns]')

sales_weather = pd.merge(sales_cost, weather, how ='inner', on =['store_id','transaction_date'])
print(sales_weather.head())

sales_weather.to_excel('sales_weather.xlsx', index=False)

   Unnamed: 0  transaction_id transaction_date transaction_time  \
0       91380           95962       2023-08-12         17:49:10   
1       22071           84940       2023-02-24         10:16:16   
2       70064           24938       2023-06-22         08:30:29   
3       71518           93910       2023-06-25         14:50:53   
4       10132           62537       2023-01-25         18:42:51   

   transaction_qty  store_id   store_location  product_id  unit_price  \
0                1         8   Hell's Kitchen          35        3.10   
1                1         5  Lower Manhattan          50        2.50   
2                1         5  Lower Manhattan          45        3.00   
3                1         5  Lower Manhattan          69        3.25   
4                1         3          Astoria          32        3.00   

  product_category           product_type            product_detail  \
0           Coffee  Premium brewed coffee  Jamaican Coffee River Rg   
1              T